In [2]:
from pwn import *
import galois
import numpy as np

In [ ]:
from pwn import *
import numpy as np

# Connect to the server
host = '10.21.235.223'
port = 8765

# Initialize an empty list to store ciphertexts
ciphertexts = []

# Open a text file to store the ciphertexts
with open('ciphertexts.txt', 'w') as file:
    for x in range(256):
        # Format the input string
        input_str = f'{x:02x}' + '000000000000000000000000000000'
        
        # Connect to the server
        conn = remote(host, port)
        
        # Send the input text
        conn.recvuntil(b'Enter plain text as hex string : ')
        conn.sendline(input_str.encode())
        
        # Receive and parse the output
        output = conn.recvall().decode()
        
        # Extract the ciphertext from the output
        lines = output.split('\n')
        for line in lines:
            if 'ciphertext' in line:
                ciphertext = line.split(': ')[1]
                ciphertexts.append(ciphertext)
                file.write(f'{input_str} : {ciphertext}\n')
        
        # Close the connection
        conn.close()

# Convert the list of ciphertexts to a NumPy array
ciphertexts_array = np.array(ciphertexts)

# Save the NumPy array to a file
np.save('ciphertexts.npy', ciphertexts_array)

print("Ciphertexts have been saved to 'ciphertexts.txt' and 'ciphertexts.npy'")

In [22]:

# Assuming you have the 256 ciphertexts stored in a list
ciphertexts = [
    "7d85e84c584131361767d3714660ebf1",
]


# Initialize an empty list to store the XORed results
xor_results = []

# Iterate over x1 from 00 to ff
for x1 in range(256):
    # Format the key with x1 and x2=x3=x4=0000
    key = f'c7c1{x1:04x}28c500004c8e000064ce0000'
    key_bytes = bytes.fromhex(key)
    
    # XOR each ciphertext with the key
    for ct in ciphertexts:
        ciphertext_bytes = bytes.fromhex(ct)
        xor_result = bytes(a ^ b for a, b in zip(ciphertext_bytes, key_bytes))
        xor_results.append(xor_result)

# Convert the list of XORed results to a NumPy array
xor_results_array = np.array([list(ct) for ct in xor_results])

# Convert each XOR result to a 4x4 matrix column-wise and store in a list
matrices = [ct.reshape(4, 4, order='F') for ct in xor_results_array]

# Print the resulting matrices
for i, matrix in enumerate(matrices):
    print(f"Resulting 4x4 matrix for x1={i:02x}:")
    print(matrix)

# # Save the matrices to a file
# with open('resulting_matrices.txt', 'w') as file:
#     for i, matrix in enumerate(matrices):
#         file.write(f"Resulting 4x4 matrix for x1={i:02x}:\n")
#         np.savetxt(file, matrix, fmt='%02x')
#         file.write('\n')

print("The resulting matrices have been saved to 'resulting_matrices.txt'")


Resulting 4x4 matrix for x1=00:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 76  54 113 241]]
Resulting 4x4 matrix for x1=01:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 77  54 113 241]]
Resulting 4x4 matrix for x1=02:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 78  54 113 241]]
Resulting 4x4 matrix for x1=03:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 79  54 113 241]]
Resulting 4x4 matrix for x1=04:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 72  54 113 241]]
Resulting 4x4 matrix for x1=05:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 73  54 113 241]]
Resulting 4x4 matrix for x1=06:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 74  54 113 241]]
Resulting 4x4 matrix for x1=07:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 75  54 113 241]]
Resulting 4x4 matrix for x1=08:
[[186 112  91  34]
 [ 68 132 233 174]
 [232  49 211 235]
 [ 68  54 113 241]]
Resulting 4x4 matri

In [14]:
x1 = 32
key = f'c7c1{x1:04x}28c500004c8e000064ce0000'
key_bytes = bytes.fromhex(key)

In [15]:
key_bytes

b'\xc7\xc1\x00 (\xc5\x00\x00L\x8e\x00\x00d\xce\x00\x00'

In [23]:
# Define the Galois Field GF(2^8)
GF = galois.GF(2**8)

reversed_mix_column=[]
for matrice in matrices:
    temp= GF(matrice)
    if np.linalg.det(temp)!=0:
        reversed_mix_column.append(np.linalg.inv(temp))

In [25]:

for i in reversed_mix_column:
    print("Matrix are")
    print(i)

Matrix are
[[204  29  60 254]
 [  7  56 234  19]
 [219  30 181 222]
 [144 183 197 186]]
Matrix are
[[166 254 213 252]
 [ 66 124  89 113]
 [200  75 227  42]
 [170 250  60 158]]
Matrix are
[[ 75 185  23 132]
 [ 53  55  66  97]
 [107 219  55  62]
 [ 27 112  25 189]]
Matrix are
[[ 10 220   2 205]
 [216 112 128  25]
 [250  68  75  88]
 [253 164 126 203]]
Matrix are
[[190  56  38   6]
 [224 163  42 166]
 [204 106  69   1]
 [  7 151  76 108]]
Matrix are
[[236   8 101 187]
 [105 250 165  58]
 [ 32  98  32 186]
 [234 208 142  20]]
Matrix are
[[195 175  63 219]
 [  5 166 185 136]
 [ 87  41  59 170]
 [126  33 243 154]]
Matrix are
[[211  43 157 119]
 [ 50 199  16  18]
 [164  23 255 152]
 [ 67 156  88 205]]
Matrix are
[[246  80 197 218]
 [169 132 110 185]
 [208 141  16 208]
 [ 99 137   1 136]]
Matrix are
[[216 184  56 121]
 [244   6  46  33]
 [221 161  64 110]
 [229  98 141  49]]
Matrix are
[[ 48 145 251 233]
 [118 204   4 179]
 [241 215 238  86]
 [ 14 154 186 249]]
Matrix are
[[ 45  57   9 251]
 [

In [26]:
State= np.array
def _shift_rows(state: State) -> State:
    new_state = np.array(state, dtype=int)
    for i in range(4):
        new_state[i] = np.roll(new_state[i], i)
    return new_state

In [27]:
reversed_shiftRow=[]

for matrice in reversed_mix_column:
    reversed_shiftRow.append(_shift_rows(matrice))
for i in reversed_shiftRow:
    print("Revered shift rows are")
    print(i)

Revered shift rows are
[[204  29  60 254]
 [ 56 234  19   7]
 [181 222 219  30]
 [186 144 183 197]]
Revered shift rows are
[[166 254 213 252]
 [124  89 113  66]
 [227  42 200  75]
 [158 170 250  60]]
Revered shift rows are
[[ 75 185  23 132]
 [ 55  66  97  53]
 [ 55  62 107 219]
 [189  27 112  25]]
Revered shift rows are
[[ 10 220   2 205]
 [112 128  25 216]
 [ 75  88 250  68]
 [203 253 164 126]]
Revered shift rows are
[[190  56  38   6]
 [163  42 166 224]
 [ 69   1 204 106]
 [108   7 151  76]]
Revered shift rows are
[[236   8 101 187]
 [250 165  58 105]
 [ 32 186  32  98]
 [ 20 234 208 142]]
Revered shift rows are
[[195 175  63 219]
 [166 185 136   5]
 [ 59 170  87  41]
 [154 126  33 243]]
Revered shift rows are
[[211  43 157 119]
 [199  16  18  50]
 [255 152 164  23]
 [205  67 156  88]]
Revered shift rows are
[[246  80 197 218]
 [132 110 185 169]
 [ 16 208 208 141]
 [136  99 137   1]]
Revered shift rows are
[[216 184  56 121]
 [  6  46  33 244]
 [ 64 110 221 161]
 [ 49 229  98 141]]


In [30]:
S_BOX = {
    0: 99,
    1: 124,
    2: 119,
    3: 123,
    4: 242,
    5: 107,
    6: 111,
    7: 197,
    8: 48,
    9: 1,
    10: 103,
    11: 43,
    12: 254,
    13: 215,
    14: 171,
    15: 118,
    16: 202,
    17: 130,
    18: 201,
    19: 125,
    20: 250,
    21: 89,
    22: 71,
    23: 240,
    24: 173,
    25: 212,
    26: 162,
    27: 175,
    28: 156,
    29: 164,
    30: 114,
    31: 192,
    32: 183,
    33: 253,
    34: 147,
    35: 38,
    36: 54,
    37: 63,
    38: 247,
    39: 204,
    40: 52,
    41: 165,
    42: 229,
    43: 241,
    44: 113,
    45: 216,
    46: 49,
    47: 21,
    48: 4,
    49: 199,
    50: 35,
    51: 195,
    52: 24,
    53: 150,
    54: 5,
    55: 154,
    56: 7,
    57: 18,
    58: 128,
    59: 226,
    60: 235,
    61: 39,
    62: 178,
    63: 117,
    64: 9,
    65: 131,
    66: 44,
    67: 26,
    68: 27,
    69: 110,
    70: 90,
    71: 160,
    72: 82,
    73: 59,
    74: 214,
    75: 179,
    76: 41,
    77: 227,
    78: 47,
    79: 132,
    80: 83,
    81: 209,
    82: 0,
    83: 237,
    84: 32,
    85: 252,
    86: 177,
    87: 91,
    88: 106,
    89: 203,
    90: 190,
    91: 57,
    92: 74,
    93: 76,
    94: 88,
    95: 207,
    96: 208,
    97: 239,
    98: 170,
    99: 251,
    100: 67,
    101: 77,
    102: 51,
    103: 133,
    104: 69,
    105: 249,
    106: 2,
    107: 127,
    108: 80,
    109: 60,
    110: 159,
    111: 168,
    112: 81,
    113: 163,
    114: 64,
    115: 143,
    116: 146,
    117: 157,
    118: 56,
    119: 245,
    120: 188,
    121: 182,
    122: 218,
    123: 33,
    124: 16,
    125: 255,
    126: 243,
    127: 210,
    128: 205,
    129: 12,
    130: 19,
    131: 236,
    132: 95,
    133: 151,
    134: 68,
    135: 23,
    136: 196,
    137: 167,
    138: 126,
    139: 61,
    140: 100,
    141: 93,
    142: 25,
    143: 115,
    144: 96,
    145: 129,
    146: 79,
    147: 220,
    148: 34,
    149: 42,
    150: 144,
    151: 136,
    152: 70,
    153: 238,
    154: 184,
    155: 20,
    156: 222,
    157: 94,
    158: 11,
    159: 219,
    160: 224,
    161: 50,
    162: 58,
    163: 10,
    164: 73,
    165: 6,
    166: 36,
    167: 92,
    168: 194,
    169: 211,
    170: 172,
    171: 98,
    172: 145,
    173: 149,
    174: 228,
    175: 121,
    176: 231,
    177: 200,
    178: 55,
    179: 109,
    180: 141,
    181: 213,
    182: 78,
    183: 169,
    184: 108,
    185: 86,
    186: 244,
    187: 234,
    188: 101,
    189: 122,
    190: 174,
    191: 8,
    192: 186,
    193: 120,
    194: 37,
    195: 46,
    196: 28,
    197: 166,
    198: 180,
    199: 198,
    200: 232,
    201: 221,
    202: 116,
    203: 31,
    204: 75,
    205: 189,
    206: 139,
    207: 138,
    208: 112,
    209: 62,
    210: 181,
    211: 102,
    212: 72,
    213: 3,
    214: 246,
    215: 14,
    216: 97,
    217: 53,
    218: 87,
    219: 185,
    220: 134,
    221: 193,
    222: 29,
    223: 158,
    224: 225,
    225: 248,
    226: 152,
    227: 17,
    228: 105,
    229: 217,
    230: 142,
    231: 148,
    232: 155,
    233: 30,
    234: 135,
    235: 233,
    236: 206,
    237: 85,
    238: 40,
    239: 223,
    240: 140,
    241: 161,
    242: 137,
    243: 13,
    244: 191,
    245: 230,
    246: 66,
    247: 104,
    248: 65,
    249: 153,
    250: 45,
    251: 15,
    252: 176,
    253: 84,
    254: 187,
    255: 22,
}
inverseSbox= {}
for key, value in S_BOX.items():
    inverseSbox[value]= key

In [34]:
transformed_matrices = []

# Iterate over each matrix
for matrix in reversed_shiftRow:
    # Create a new matrix with the same shape
    transformed_matrix = np.zeros_like(matrix)
    
    # Iterate over each element in the matrix
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            # Replace the value with the corresponding inverseSbox value
            transformed_matrix[i, j] = inverseSbox[matrix[i, j]]
    
    # Append the transformed matrix to the new list
    transformed_matrices.append(transformed_matrix)

# Print the transformed matrices
for i, matrix in enumerate(transformed_matrices):
    print(f"Transformed matrix {i+1}:")
    print(matrix)

Transformed matrix 1:
[[ 39 222 109  12]
 [118 187 130  56]
 [210 156 159 233]
 [192 150  32   7]]
Transformed matrix 2:
[[197  12 181  85]
 [  1  21  44 246]
 [ 77 149 177 204]
 [223  98  20 109]]
Transformed matrix 3:
[[204 219 135  79]
 [178 246 216 217]
 [178 209   5 159]
 [205  68 208 142]]
Transformed matrix 4:
[[163 147 106 128]
 [208  58 142  45]
 [204  94  20 134]
 [ 89  33  29 138]]
Transformed matrix 5:
[[ 90 118  35 165]
 [113 149 197 160]
 [104   9  39  88]
 [184  56 133  93]]
Transformed matrix 6:
[[131 191 188 254]
 [ 20  41 162 228]
 [ 84 192  84 171]
 [155 187  96 230]]
Transformed matrix 7:
[[ 51  27  37 159]
 [197 219 151  54]
 [ 73  98 218  76]
 [ 55 138 123 126]]
Transformed matrix 8:
[[169  11 117   2]
 [ 49 124  57 161]
 [125 226  29 135]
 [128 100  28  94]]
Transformed matrix 9:
[[214 108   7 122]
 [ 79  69 219 183]
 [124  96  96 180]
 [151   0 242   9]]
Transformed matrix 10:
[[ 45 154 118 175]
 [165 195 123 186]
 [114  69 201 241]
 [ 46  42 171 180]]
Transform

In [3]:
State= np.array
def reverse_mix_column(state: State)->State:
    GF = galois.GF(2**8)
    return np.linalg.inv(GF(state))

State= np.array
def _shift_rows(state: State) -> State:
    new_state = np.array(state, dtype=int)
    for i in range(4):
        new_state[i] = np.roll(new_state[i], i)
    return new_state

def inverse_SBOX(state: State)->State:
    toret= np.zeros_like(state)
    for i in range(state.shape[0]):
        for j in range(state.shape[1]):
            # Replace the value with the corresponding inverseSbox value
            toret[i, j] = inverseSbox[state[i, j]]
    return toret


In [ ]:
#guessing one x1
valueforx1:int
for x1 in range(0,256*256):
    key=f'c7c1{x1:04x}28c500004c8e000064ce0000'
    key_bytes = bytes.fromhex(key)

    # XOR each ciphertext with the key
    for ct in ciphertexts:
        ciphertext_bytes = bytes.fromhex(ct)
        xor_result = bytes(a ^ b for a, b in zip(ciphertext_bytes, key_bytes))
        xor_results.append(xor_result)
    #here we got 256 cipher text
    #now from here we will convert from one line string to matrices
    xor_results_array= np.array([list(ct) for ct in xor_results])

    after_mix_column= [ct.reshape(4, 4, order='F') for ct in xor_results_array]
    #now for a pariticular x1 we have all the possible cyphertexts here
    #now we need to reverse the mix column

    reverse_mix_column=[]
    for matrice in after_mix_column:
        if np.linalg.det(matrice)!=0:
            reverse_mix_column.append(reverse_mix_column(matrice))
    #now we have reversed the mix columns, now our task is to find the
    #reverse shift bytes

    reversed_shiftRow = []
    for matrice in reverse_mix_column:
        reversed_shiftRow.append(_shift_rows(matrice))

    #after getting the shift rows, i need to reverse the subbytes
    reverse_subbytes = []
    for matrice in reversed_shiftRow:
        reverse_subbytes.append(inverse_SBOX(matrice))

    # here we got the reverse_subbytes. Now at this point i need to to
    # xor of every correspoinding element ie.

    val1=0
    val2=0
    val3=0
    val4=0
    for matrice in reverse_subbytes:
        val1 ^= matrice[0][0]
        val2 ^= matrice[1][3]
        val3 ^= matrice[2][2]
        val4 ^= matrice[3][1]
    if(val1==0 and val2==0 and val3==0 and val4==0):
        valueforx1= x1
        break

In [ ]:
LOOKUP_TABLE = [
    0x8D,
    0x01,
    0x02,
    0x04,
    0x08,
    0x10,
    0x20,
    0x40,
    0x80,
    0x1B,
    0x36,
    0x6C,
    0xD8,
    0xAB,
    0x4D,
    0x9A,
    0x2F,
    0x5E,
    0xBC,
    0x63,
    0xC6,
    0x97,
    0x35,
    0x6A,
    0xD4,
    0xB3,
    0x7D,
    0xFA,
    0xEF,
    0xC5,
    0x91,
    0x39,
    0x72,
    0xE4,
    0xD3,
    0xBD,
    0x61,
    0xC2,
    0x9F,
    0x25,
    0x4A,
    0x94,
    0x33,
    0x66,
    0xCC,
    0x83,
    0x1D,
    0x3A,
    0x74,
    0xE8,
    0xCB,
    0x8D,
    0x01,
    0x02,
    0x04,
    0x08,
    0x10,
    0x20,
    0x40,
    0x80,
    0x1B,
    0x36,
    0x6C,
    0xD8,
    0xAB,
    0x4D,
    0x9A,
    0x2F,
    0x5E,
    0xBC,
    0x63,
    0xC6,
    0x97,
    0x35,
    0x6A,
    0xD4,
    0xB3,
    0x7D,
    0xFA,
    0xEF,
    0xC5,
    0x91,
    0x39,
    0x72,
    0xE4,
    0xD3,
    0xBD,
    0x61,
    0xC2,
    0x9F,
    0x25,
    0x4A,
    0x94,
    0x33,
    0x66,
    0xCC,
    0x83,
    0x1D,
    0x3A,
    0x74,
    0xE8,
    0xCB,
    0x8D,
    0x01,
    0x02,
    0x04,
    0x08,
    0x10,
    0x20,
    0x40,
    0x80,
    0x1B,
    0x36,
    0x6C,
    0xD8,
    0xAB,
    0x4D,
    0x9A,
    0x2F,
    0x5E,
    0xBC,
    0x63,
    0xC6,
    0x97,
    0x35,
    0x6A,
    0xD4,
    0xB3,
    0x7D,
    0xFA,
    0xEF,
    0xC5,
    0x91,
    0x39,
    0x72,
    0xE4,
    0xD3,
    0xBD,
    0x61,
    0xC2,
    0x9F,
    0x25,
    0x4A,
    0x94,
    0x33,
    0x66,
    0xCC,
    0x83,
    0x1D,
    0x3A,
    0x74,
    0xE8,
    0xCB,
    0x8D,
    0x01,
    0x02,
    0x04,
    0x08,
    0x10,
    0x20,
    0x40,
    0x80,
    0x1B,
    0x36,
    0x6C,
    0xD8,
    0xAB,
    0x4D,
    0x9A,
    0x2F,
    0x5E,
    0xBC,
    0x63,
    0xC6,
    0x97,
    0x35,
    0x6A,
    0xD4,
    0xB3,
    0x7D,
    0xFA,
    0xEF,
    0xC5,
    0x91,
    0x39,
    0x72,
    0xE4,
    0xD3,
    0xBD,
    0x61,
    0xC2,
    0x9F,
    0x25,
    0x4A,
    0x94,
    0x33,
    0x66,
    0xCC,
    0x83,
    0x1D,
    0x3A,
    0x74,
    0xE8,
    0xCB,
    0x8D,
    0x01,
    0x02,
    0x04,
    0x08,
    0x10,
    0x20,
    0x40,
    0x80,
    0x1B,
    0x36,
    0x6C,
    0xD8,
    0xAB,
    0x4D,
    0x9A,
    0x2F,
    0x5E,
    0xBC,
    0x63,
    0xC6,
    0x97,
    0x35,
    0x6A,
    0xD4,
    0xB3,
    0x7D,
    0xFA,
    0xEF,
    0xC5,
    0x91,
    0x39,
    0x72,
    0xE4,
    0xD3,
    0xBD,
    0x61,
    0xC2,
    0x9F,
    0x25,
    0x4A,
    0x94,
    0x33,
    0x66,
    0xCC,
    0x83,
    0x1D,
    0x3A,
    0x74,
    0xE8,
    0xCB,
    0x8D,
]

In [ ]:
def xor(w1: bytes, w2: bytes) -> bytes:
    return bytes([i ^ j for i, j in zip(w1, w2)])

def inv_rot_word(word: bytes) -> bytes:
    if len(word) != 4:
        raise ValueError(f"Wrong word length: {len(word)}")
    return bytes([word[-1]]) + word[:-1]

def inv_sub_word(word: bytes) -> bytes:
    if len(word) != 4:
        raise ValueError(f"Wrong word length: {len(word)}")
    return bytes([inverseSbox[b] for b in word])

def _r_con(n: int) -> bytes:
    if not 0 <= n < 256:
        raise ValueError(f"Wrong n: {n}")
    return bytes([LOOKUP_TABLE[n], 0, 0, 0])

def reverse_key_expansion(last_round_key: bytes, rounds: int) -> bytes:
    if len(last_round_key) != 16:
        raise ValueError(f"Wrong key length: {len(last_round_key)}")
    
    expanded_key = [last_round_key[i:i+4] for i in range(0, len(last_round_key), 4)]
    
    for r in range(rounds, 0, -1):
        temp = expanded_key[0]
        temp = inv_rot_word(temp)
        temp = inv_sub_word(temp)
        temp = xor(temp, _r_con(r))
        expanded_key = [xor(temp, expanded_key[-1])] + expanded_key[:-1]
        for i in range(1, 4):
            expanded_key[i] = xor(expanded_key[i], expanded_key[i-1])
    
    original_key = b''.join(expanded_key)
    return original_key

# Example usage
last_round_key = b'\xd0\x14\xf9\xa8\xc9\xee\x25\x89\xe1\x3f\x0c\xc8\xb6\x63\x0c\xa6'
rounds = 10
original_key = reverse_key_expansion(last_round_key, rounds)
print(f"Last Round Key: {last_round_key.hex()}")
print(f"Original Key: {original_key.hex()}")
